In [7]:
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure, OperationFailure


class DatabaseConnection:
    def __init__(self,username, password, host='localhost', port='27017'):
        # Credenciales del docker-compose
        username = username
        password = password
        host = host
        port = port
        
        # URI completa con autenticación
        self.uri = f'mongodb://{username}:{password}@{host}:{port}/'
        self.client = None
        self.db = None
    
    def connect(self, db_name):
        try:
            self.client = MongoClient(
                self.uri,
                serverSelectionTimeoutMS=5000,
                authSource='admin'  # Importante: autenticarse contra la DB 'admin'
            )
            
            # Verificar conexión
            self.client.admin.command('ping')
            self.db = self.client[db_name]
            print(f"✓ Conectado a la base de datos '{db_name}'")
            return self.db
            
        except ConnectionFailure as e:
            print(f"No se pudo conectar a MongoDB: {e}")
            return None
        except OperationFailure as e:
            print(f"Error de autenticación: {e}")
            return None
    
    def close(self):
        if self.client:
            self.client.close()
            print("Conexión cerrada")
# Uso


In [ ]:
db_conn = DatabaseConnection()
db = db_conn.connect('universidad')

try:
    resultado = db.estudiantes.insert_one({"nombre": "Juan", "edad": 25})
    print(f"✓ Documento insertado: {resultado.inserted_id}")
except Exception as e:
    print(f"❌ ERROR al insertar: {e}")
    # Probablemente verás: "command insert requires authentication"


✓ Conectado a la base de datos 'universidad'
✓ Documento insertado: 6984c31fd0178b3e06025f06


In [12]:
from bson import ObjectId

# MongoDB genera automáticamente un _id si no lo proporcionas
doc1 = {"nombre": "Juan"}
db.estudiantes.insert_one(doc1)
print(doc1["_id"])  # ObjectId('65f1a2b3c4d5e6f7g8h9i0j1')

# Puedes especificar tu propio _id
doc2 = {
    "_id": "EST001",  # Puede ser string, int, ObjectId...
    "nombre": "Ana"
}
db.estudiantes.insert_one(doc2)

# ObjectId contiene información temporal
oid = ObjectId()
print(f"Timestamp: {oid.generation_time}")  # Fecha de creación

6984c655d0178b3e06025f08
Timestamp: 2026-02-05 16:33:25+00:00


In [15]:
from datetime import datetime

In [17]:
estudiantes = db.estudiantes

resultado = estudiantes.insert_one({
    "nombre": "Carlos Ruiz",
    "email": "carlos@example.com",
    "edad": 24,
    "fecha_registro": datetime.now()
})

print(f"ID insertado: {resultado.inserted_id}")
print(f"¿Insertado? {resultado.acknowledged}")

# insert_many() - insertar múltiples documentos
documentos = [
    {"nombre": "Ana López", "email": "ana@example.com", "edad": 22},
    {"nombre": "Luis Martín", "email": "luis@example.com", "edad": 23}
]

resultado = estudiantes.insert_many(documentos)
print(f"IDs insertados: {resultado.inserted_ids}")

ID insertado: 6984c75ad0178b3e06025f0a
¿Insertado? True
IDs insertados: [ObjectId('6984c75ad0178b3e06025f0b'), ObjectId('6984c75ad0178b3e06025f0c')]


In [18]:
from bson import ObjectId, Decimal128
from datetime import datetime
import uuid

# Documento con todos los tipos comunes
documento_completo = {
    # Tipos básicos
    "_id": ObjectId(),                    # ObjectId único
    "nombre": "María García",             # String
    "edad": 28,                           # Int32
    "activo": True,                       # Boolean
    "salario": 45000.50,                  # Double (float)
    "salario_preciso": Decimal128("45000.50"),  # Decimal128 (precisión)
    
    # Fechas y tiempo
    "fecha_nacimiento": datetime(1995, 3, 15),
    "fecha_registro": datetime.now(),
    
    # Identificadores
    "uuid": uuid.uuid4(),                 # UUID
    
    # Valores nulos
    "telefono_secundario": None,          # Null
    
    # Arrays
    "hobbies": ["lectura", "natación", "música"],
    "calificaciones": [8.5, 9.0, 7.5, 9.5],
    
    # Subdocumentos (objetos anidados)
    "direccion": {
        "calle": "Calle Mayor 10",
        "ciudad": "Valencia",
        "codigo_postal": "46001",
        "coordenadas": {
            "lat": 39.4699,
            "lon": -0.3763
        }
    },
    
    # Array de subdocumentos
    "contactos": [
        {"tipo": "email", "valor": "maria@example.com"},
        {"tipo": "telefono", "valor": "+34612345678"}
    ],
    
    # Binario
    "avatar": b'\x89PNG\r\n\x1a\n...',    # Binary data
}

estudiantes.insert_one(documento_completo)

ValueError: cannot encode native uuid.UUID with UuidRepresentation.UNSPECIFIED. UUIDs can be manually converted to bson.Binary instances using bson.Binary.from_uuid() or a different UuidRepresentation can be configured. See the documentation for UuidRepresentation for more information.

In [6]:

from pymongo import MongoClient
from pymongo.errors import ConnectionFailure, OperationFailure


class DatabaseConnection:
    def __init__(self,username, password, host='localhost', port='27017'):
        # Credenciales del docker-compose
        username = username
        password = password
        host = host
        port = port
        
        # URI completa con autenticación
        self.uri = f'mongodb://{username}:{password}@{host}:{port}/'
        self.client = None
        self.db = None
    
    def connect(self, db_name):
        try:
            self.client = MongoClient(
                self.uri,
                serverSelectionTimeoutMS=5000,
                authSource='admin',  # Importante: autenticarse contra la DB 'admin',
                uuidRepresentation='standard'  # Para UUIDs
            )
            
            # Verificar conexión
            self.client.admin.command('ping')
            self.db = self.client[db_name]
            print(f"✓ Conectado a la base de datos '{db_name}'")
            return self.db
            
        except ConnectionFailure as e:
            print(f"No se pudo conectar a MongoDB: {e}")
            return None
        except OperationFailure as e:
            print(f"Error de autenticación: {e}")
            return None
    
    def close(self):
        if self.client:
            self.client.close()
            print("Conexión cerrada")
# Uso
db_conn = DatabaseConnection("admin","password123")
db = db_conn.connect('universidad')
estudiantes = db.estudiantes

✓ Conectado a la base de datos 'universidad'


In [4]:
from bson import ObjectId, Decimal128
from datetime import datetime
import uuid

# Documento con todos los tipos comunes
documento_completo = {
    # Tipos básicos
    "_id": ObjectId(),                    # ObjectId único
    "nombre": "María García",             # String
    "edad": 28,                           # Int32
    "activo": True,                       # Boolean
    "salario": 45000.50,                  # Double (float)
    "salario_preciso": Decimal128("45000.50"),  # Decimal128 (precisión)
    
    # Fechas y tiempo
    "fecha_nacimiento": datetime(1995, 3, 15),
    "fecha_registro": datetime.now(),
    
    # Identificadores
    "uuid": uuid.uuid4(),                 # UUID
    
    # Valores nulos
    "telefono_secundario": None,          # Null
    
    # Arrays
    "hobbies": ["lectura", "natación", "música"],
    "calificaciones": [8.5, 9.0, 7.5, 9.5],
    
    # Subdocumentos (objetos anidados)
    "direccion": {
        "calle": "Calle Mayor 10",
        "ciudad": "Valencia",
        "codigo_postal": "46001",
        "coordenadas": {
            "lat": 39.4699,
            "lon": -0.3763
        }
    },
    
    # Array de subdocumentos
    "contactos": [
        {"tipo": "email", "valor": "maria@example.com"},
        {"tipo": "telefono", "valor": "+34612345678"}
    ],
    
    # Binario
    "avatar": b'\x89PNG\r\n\x1a\n...',    # Binary data
}
db_conn = DatabaseConnection('admin','password123')
db = db_conn.connect('universidad')
estudiantes = db.estudiantes
estudiantes.insert_one(documento_completo)

✓ Conectado a la base de datos 'universidad'


InsertOneResult(ObjectId('6986195bb33f1adf1c866bd4'), acknowledged=True)

In [ ]:
from pymongo.errors import DuplicateKeyError, BulkWriteError

# Ejemplo 1: Error por _id duplicado
try:
    estudiantes.insert_one({"_id": "EST001", "nombre": "Carlos"})
    estudiantes.insert_one({"_id": "EST001", "nombre": "Ana"})  # Error
except DuplicateKeyError as e:
    print(f"Error: El _id ya existe - {e}")

# Ejemplo 2: Inserción masiva con manejo de errores
documentos = [
    {"_id": 1, "nombre": "Usuario1"},
    {"_id": 2, "nombre": "Usuario2"},
    {"_id": 1, "nombre": "Duplicado"},  # Este fallará
    {"_id": 3, "nombre": "Usuario3"}
]

try:
    # ordered=False permite continuar después de un error
    resultado = estudiantes.insert_many(documentos, ordered=False)
except BulkWriteError as e:
    print(f"Algunos documentos no se insertaron: {e}")
    print(f"Insertados exitosamente: {e.details['nInserted']}")

# Ejemplo 3: Validación antes de insertar
def insertar_estudiante(nombre, email, edad):
    # Validaciones básicas
    if not nombre or len(nombre) < 2:
        return {"error": "Nombre inválido"}
    
    if not email or "@" not in email:
        return {"error": "Email inválido"}
    
    if edad < 16 or edad > 100:
        return {"error": "Edad fuera de rango"}
    
    # Verificar email duplicado
    if estudiantes.find_one({"email": email}):
        return {"error": "Email ya registrado"}
    
    try:
        resultado = estudiantes.insert_one({
            "nombre": nombre,
            "email": email,
            "edad": edad,
            "fecha_registro": datetime.now()
        })
        return {"success": True, "id": resultado.inserted_id}
    except Exception as e:
        return {"error": str(e)}

# Uso
print(insertar_estudiante("María", "maria@example.com", 22))

Error: El _id ya existe - E11000 duplicate key error collection: universidad.estudiantes index: _id_ dup key: { _id: "EST001" }, full error: {'index': 0, 'code': 11000, 'errmsg': 'E11000 duplicate key error collection: universidad.estudiantes index: _id_ dup key: { _id: "EST001" }', 'keyPattern': {'_id': 1}, 'keyValue': {'_id': 'EST001'}}


BulkWriteError: batch op errors occurred, full error: {'writeErrors': [{'index': 2, 'code': 11000, 'errmsg': 'E11000 duplicate key error collection: universidad.estudiantes index: _id_ dup key: { _id: 1 }', 'keyPattern': {'_id': 1}, 'keyValue': {'_id': 1}, 'op': {'_id': 1, 'nombre': 'Duplicado'}}], 'writeConcernErrors': [], 'nInserted': 3, 'nUpserted': 0, 'nMatched': 0, 'nModified': 0, 'nRemoved': 0, 'upserted': []}